In [1]:
from micromind import MicroMind, Metric
from micromind.networks import PhiNet
from micromind.utils.parse import parse_arguments

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from dataset import ImageWoof
from matplotlib import pyplot as plt

from torchinfo import summary

batch_size = 64


In [2]:
#importing libraries

import os
import numpy as np
import torch
import time
import torch.utils.data
from torch.utils.data.dataset import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import random
import cv2
import glob
from PIL import Image
import ntpath
import os
from tqdm.notebook import tqdm
%matplotlib inline

In [3]:
def START_seed():
    seed = 9
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

lr = 0.1
epochs=50
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_batch_size = 16
test_batch_size = 16

In [4]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 8, 7, 1),
            nn.ReLU(),
            nn.Conv2d(8, 8, 7, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            #####
            nn.Conv2d(8, 16, 5, 1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 5, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            #####
            nn.Conv2d(16, 32, 3, 1),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, 1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(
                output_size=8,
            ),
        )
        self.classifier = nn.Sequential(
            nn.Linear(32 * 8 * 8, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        features = self.feature_extractor(x)
        features = torch.flatten(features, start_dim=1)
        out = self.classifier(features)
        return out

    def num_of_params(self):
        total = 0
        for layer_params in self.feature_extractor.parameters():
            total += layer_params.numel()
        for layer_params in self.classifier.parameters():
            total += layer_params.numel()
        return total

    def compute_loss(self, pred, batch):
        return nn.CrossEntropyLoss()(pred, batch[1])

In [8]:
if __name__ == "__main__":

    m = CNN(num_classes=10)

    def compute_accuracy(pred, batch):
        tmp = (pred.argmax(1) == batch[1]).float()
        return tmp

    transform = transforms.Compose(
        [transforms.ToTensor(), 
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
         transforms.Resize((160, 160), antialias=True)
        ]
    )

    trainset = ImageWoof(
        root=".", train=True, transform=transform, img_size=160
    )
    testset = ImageWoof(
        root=".", train=False, transform=transform, img_size=160
    )

    ## split into train, val, test 
    print(len(trainset))     
    val_size = int(0.1 * len(trainset))
    print(val_size)
    train_size = len(trainset) - val_size
    train, val = torch.utils.data.random_split(trainset, [train_size, val_size])    

    trainloader = torch.utils.data.DataLoader(
        train, batch_size=batch_size, shuffle=True, num_workers=1
    )
    valloader = torch.utils.data.DataLoader(
        val, batch_size=batch_size, shuffle=False, num_workers=1
    )    
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=batch_size, shuffle=False, num_workers=1
    )

    print("Trainset size: ", len(train)//batch_size)
    print("Valset size: ", len(val)//batch_size)
    print("Testset size: ", len(testset)//batch_size)

    acc = Metric(name="accuracy", fn=compute_accuracy)


9025
902
Trainset size:  126
Valset size:  14
Testset size:  61


In [10]:
batch = next(iter(trainloader))
data = batch[0]
target = batch[1]

target = target.to('cuda:0')

print(batch[0].shape)
print(len(batch))

print(type(batch))

out = m.forward(batch[0].to('cuda:0'))

print(out)

print(out.shape)

print(torch.argmax(out, dim=1))
print(target)

corrects = target == torch.argmax(out, dim=1)

accuracy = torch.sum(corrects) / len(corrects)

print(accuracy)


torch.Size([64, 3, 160, 160])
2
<class 'list'>


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [67]:
START_seed()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(m.parameters(), lr=lr) 

In [73]:
# ## Creating training loop
def train(model):    
    running_loss = 0.0    

    model.train()

    pbar = tqdm(total=len(trainloader), dynamic_ncols=True)
    
    for batch, data in enumerate(trainloader, 0):       

        # zero the parameter gradients
        optimizer.zero_grad()    

        pbar.update(1)

        # get the inputs; data is a list of [inputs, labels]
        inputs, targets = data

        # forward + backward + optimize
        outputs = model(inputs.to(device))        
        loss = criterion(outputs, targets.to(device))          
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
        # Postfix will be displayed on the right,
        # formatted automatically based on argument's datatype
        pbar.set_postfix(loss=running_loss/(batch+1), lr=optimizer.param_groups[0]['lr'])
       
    print(f'loss: {running_loss / (batch+1):.3f}')
    running_loss = 0.0   

In [74]:
def validate(model):       
    running_loss = 0.0    

    model.eval()

    total = 0
    correct = 0

    tmpenum = enumerate(valloader, 0)

    pbar = tqdm(tmpenum, total=len(valloader))

    for i, data in tmpenum:

        pbar.update(1)
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data     

        # forward + backward + optimize
        outputs = model(inputs.to(device))
        loss = criterion(outputs, labels.to(device))  

        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:                                
                correct +=1            
            total+=1                       
        running_loss += loss.item()        

        pbar.set_postfix(loss=running_loss/(i+1))        

        
    print(' val loss: {:.4f} accuracy: {:.4f}'.format(loss/(i+1), 100.*correct/total)) 

In [75]:
# move model to device whatever device it is
m.to(device)
start = time.time()
for epoch in range(0, epochs):
    print("Epoch number: {0}".format(epoch))
    train(m)
    validate(m)
end = time.time()
Total_time=end-start
print('Total training and inference time is: {0}'.format(Total_time))

Epoch number: 0


  0%|          | 0/64 [00:00<?, ?it/s]

loss: 2.295


  0%|          | 0/8 [00:00<?, ?it/s]

 val loss: 0.2834 accuracy: 9.5344
Epoch number: 1


...

loss: 2.294


  0%|          | 0/8 [00:00<?, ?it/s]

 val loss: 0.2833 accuracy: 9.5344
Epoch number: 17


  0%|          | 0/64 [00:00<?, ?it/s]

KeyboardInterrupt: 